# Importing Dependencies

In [1]:
import os
import string
import numpy as np
import pandas as pd
from string import digits
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline
import re
import logging
import tensorflow as tf
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
import unicodedata
import io
import time
import warnings
import sys

import os
import gdown
import pandas as pd

data_file_1 = 'Hindi_English_Truncated_Corpus.csv'

# Check if the file exists in the current directory
if not os.path.isfile(data_file_1):
    # Download the data file from Google Drive
    url = 'https://drive.google.com/uc?id=1uGPKzMUWYqDzPox6GznsT1Hx4ERpiAMo'
    output = data_file_1
    gdown.download(url, output, quiet=False)

# Read the CSV file into a DataFrame
lines = pd.read_csv(data_file_1)

Downloading...
From: https://drive.google.com/uc?id=1uGPKzMUWYqDzPox6GznsT1Hx4ERpiAMo
To: /content/Hindi_English_Truncated_Corpus.csv
100%|██████████| 41.4M/41.4M [00:00<00:00, 90.4MB/s]


In [2]:
import tensorflow as tf

if tf.executing_eagerly():
    print("Eager execution is enabled.")
else:
    print("Eager execution is not enabled.")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
logging.getLogger('tensorflow').setLevel(logging.FATAL)

Eager execution is enabled.


In [3]:
PATH = "/content/Hindi_English_Truncated_Corpus.csv"

## Preprocess English and Hindi sentences

In [4]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    w = w.rstrip().strip()
    return w

def hindi_preprocess_sentence(w):
    w = w.rstrip().strip()
    return w

In [5]:
def create_dataset(path=PATH):
    lines=pd.read_csv(path,encoding='utf-8')
    lines=lines.dropna()
    lines = lines[lines['source']=='ted']
    en = []
    hd = []
    for i, j in zip(lines['english_sentence'], lines['hindi_sentence']):
        en_1 = [preprocess_sentence(w) for w in i.split(' ')]
        en_1.append('<end>')
        en_1.insert(0, '<start>')
        hd_1 = [hindi_preprocess_sentence(w) for w in j.split(' ')]
        hd_1.append('<end>')
        hd_1.insert(0, '<start>')
        en.append(en_1)
        hd.append(hd_1)
    return hd, en

In [6]:
def max_length(tensor):
    return max(len(t) for t in tensor)

### Tokenization of the data

In [7]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
  lang_tokenizer.fit_on_texts(lang)
  tensor = lang_tokenizer.texts_to_sequences(lang)
  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,padding='post')
  return tensor, lang_tokenizer

In [8]:
def load_dataset(path=PATH):
    targ_lang, inp_lang = create_dataset(path)
    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, targ_lang_tokenizer = tokenize(targ_lang)
    return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [9]:
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(PATH)
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

### Create Train and Test dataset

In [10]:
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

31904 31904 7977 7977


In [11]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
39 ----> do
12 ----> you
81 ----> know
9 ----> that
5785 ----> bipolar
2704 ----> disorder
11 ----> is
11103 ----> nicknamed
3 ----> the
5968 ----> ceo
8511 ----> disease ?
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
41 ----> क्या
69 ----> आपको
108 ----> पता
6 ----> है
10 ----> कि
7883 ----> बाईपोलर
5747 ----> डिसार्डर
11 ----> को
10937 ----> सी.ई.ओ.
828 ----> बीमारी
26 ----> भी
212 ----> कहते
13 ----> हैं
203 ----> ?
2 ----> <end>


### Create Dataset
> We are using minimal configuration as the notebbok is not focussed on metrics performance but rather the implementation.

In [12]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 128
units = 256
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

## Encoder Decoder with Attention Model

> The Encoder-Decoder model with Attention is a versatile approach to sequence learning that doesn't rely heavily on pre-defined assumptions about the structure of the sequences. It leverages a multilayered Gated Recurrent Unit (GRU) to transform the input sequence into a fixed-sized vector representation. Subsequently, another deep GRU is used to generate the target sequence from this vector representation.
<img src="https://www.researchgate.net/profile/Vlad_Zhukov2/publication/321210603/figure/fig1/AS:642862530191361@1530281779831/An-example-of-sequence-to-sequence-model-with-attention-Calculation-of-cross-entropy.png" width="800" alt="attention mechanism">

>In a sequence-to-sequence model, there are two components: an encoder and a decoder. These components are essentially two distinct neural network models combined into a single large network. The role of the encoder is to comprehend the input sequence and generate a lower-dimensional representation of it. This representation is then passed to the decoder, which produces its own sequence as the output.

The input sequence is fed into the encoder model, which produces the encoder output. To capture the importance of each input word, the attention mechanism assigns weights to them. These weights are then utilized by the decoder to predict the next word in the sentence. In this particular case, the Bahdanau attention mechanism is employed for the encoder.



### Encoder

In [13]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

### Attention Mechanism

In [14]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    hidden_with_time_axis = tf.expand_dims(query, 1)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))
    attention_weights = tf.nn.softmax(score, axis=1)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)
    return context_vector, attention_weights

### Decoder

In [15]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    context_vector, attention_weights = self.attention(hidden, enc_output)
    x = self.embedding(x)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
    output, state = self.gru(x)
    output = tf.reshape(output, (-1, output.shape[2]))
    x = self.fc(output)
    return x, state, attention_weights

decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

### Optimizer

In [16]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)
  mask = tf.cast(mask, dtype=loss_.dtype)
#   print(type(mask))
  loss_ *= mask
  return tf.reduce_mean(loss_)

In [17]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

## Training

>1. Pass *input* through *encoder* to get *encoder output*..
>2. Then encoder output, encoder hidden state and the decoder input is passed to decoder.
>3. Decoder returns *predictions* and *decoder hidden state*.
>4. Decoder hidden state is then passed back to model.
>5. Predictions are used to calculate loss.
>6. Use *teacher forcing* (technique where the target word is passed as the next input to the decoder) for the next input to the decoder.
>7. Calculate gradients and apply it to *optimizer* for backpropogation.

In [18]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0
  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)
    # Teacher forcing
    for t in range(1, targ.shape[1]):
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
      loss += loss_function(targ[:, t], predictions)
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))
  variables = encoder.trainable_variables + decoder.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))
  return batch_loss

In [19]:
EPOCHS = 20

for epoch in range(EPOCHS):
  start = time.time()
  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0
  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss
    if batch % 100 == 0:
        print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                     batch,
                                                     batch_loss.numpy()))
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 2.8985
Epoch 1 Batch 100 Loss 1.7292
Epoch 1 Batch 200 Loss 1.8486
Epoch 1 Batch 300 Loss 1.7851
Epoch 1 Batch 400 Loss 1.8360
Epoch 1 Loss 1.9441
Time taken for 1 epoch 120.05863451957703 sec

Epoch 2 Batch 0 Loss 1.7397
Epoch 2 Batch 100 Loss 1.7423
Epoch 2 Batch 200 Loss 1.7813
Epoch 2 Batch 300 Loss 1.6997
Epoch 2 Batch 400 Loss 1.6736
Epoch 2 Loss 1.7331
Time taken for 1 epoch 43.12205672264099 sec

Epoch 3 Batch 0 Loss 1.5814
Epoch 3 Batch 100 Loss 1.6199
Epoch 3 Batch 200 Loss 1.6756
Epoch 3 Batch 300 Loss 1.5605
Epoch 3 Batch 400 Loss 1.3981
Epoch 3 Loss 1.6216
Time taken for 1 epoch 40.73565983772278 sec

Epoch 4 Batch 0 Loss 1.6252
Epoch 4 Batch 100 Loss 1.5812
Epoch 4 Batch 200 Loss 1.5103
Epoch 4 Batch 300 Loss 1.5693
Epoch 4 Batch 400 Loss 1.5740
Epoch 4 Loss 1.5263
Time taken for 1 epoch 40.38520789146423 sec

Epoch 5 Batch 0 Loss 1.5224
Epoch 5 Batch 100 Loss 1.3439
Epoch 5 Batch 200 Loss 1.5436
Epoch 5 Batch 300 Loss 1.4501
Epoch 5 Batch 400 Loss 1.

In [20]:
def evaluate(sentence):
    attention_plot = np.zeros((max_length_targ, max_length_inp))
    sentence = preprocess_sentence(sentence)
    inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                           maxlen=max_length_inp,
                                                           padding='post')
    inputs = tf.convert_to_tensor(inputs)
    result = ''
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)
    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)
        predicted_id = tf.argmax(predictions[0]).numpy()
        result += targ_lang.index_word[predicted_id] + ' '
        if targ_lang.index_word[predicted_id] == '<end>':
            return result, sentence
        dec_input = tf.expand_dims([predicted_id], 0)
    return result, sentence

In [21]:
def translate(sentence):
    result, sentence = evaluate(sentence)
    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

In [22]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [35]:
translate(u'The world is bipolar.')

Input: the world is bipolar .
Predicted translation: दुनिया में कैंसर है, <end> 
